#**Ingest**

In [ ]:
import numpy as np
import pandas as pd
from random import shuffle
from google.colab import drive
import os
import cv2
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/My Drive/Data/train.zip'
!unzip '/content/drive/My Drive/Data/test.zip'

#**EDA**

In [ ]:
train_path = "./train"
test_path = "./test"

In [ ]:
ROWS = 64
COLS = 64
CHANNELS = 3

In [ ]:
images      = [img for img in os.listdir(train_path)]
images_dog  = [img for img in os.listdir(train_path) if "dog" in img]
images_cat  = [img for img in os.listdir(train_path) if "cat" in img]

images_test = [img for img in os.listdir(test_path)]

In [ ]:
train_dog = images_dog[:800]
train_cat = images_cat[:800]
valid_dog = images_dog[800:1000]
valid_cat = images_cat[800:1000]

train_list = train_dog + train_cat
valid_list = valid_dog + valid_cat
test_list  = images_test[0:]

shuffle(train_list)

train = np.ndarray(shape=(len(train_list),ROWS, COLS))
train_color = np.ndarray(shape=(len(train_list), ROWS, COLS, CHANNELS), dtype=np.uint8)
test = np.ndarray(shape=(len(test_list),ROWS, COLS))
test_color = np.ndarray(shape=(len(images_test), ROWS, COLS, CHANNELS), dtype=np.uint8)
valid = np.ndarray(shape=(len(valid_list), ROWS, COLS))
valid_color = np.ndarray(shape=(len(valid_list), ROWS, COLS, CHANNELS), dtype=np.uint8)


In [ ]:
labels = np.ndarray(len(train_list))

for i, img_path in enumerate(train_list):
    img_color = cv2.imread(os.path.join(train_path, img_path), 1)
    img_color = cv2.resize(img_color, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    
    train[i] = img
    train_color[i] = img_color
   
    if "dog" in img_path:
        labels[i] = 0
    else:
        labels[i] = 1

In [ ]:
valid_labels = np.ndarray(len(valid_list))

for i, img_path in enumerate(valid_list):
    img_color = cv2.imread(os.path.join(train_path, img_path), 1)
    img_color = cv2.resize(img_color, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    
    valid[i] = img
    valid_color[i] = img_color
    
    if "dog" in img_path:
        valid_labels[i] = 0
    else:
        valid_labels[i] = 1

In [ ]:
for i, img_path in enumerate(test_list):
    img_color = cv2.imread(os.path.join(test_path, img_path), 1)
    img_color = cv2.resize(img_color, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    
    test[i] = img
    test_color[i] = img_color

In [ ]:
from keras.utils import np_utils

X_train = train_color / 255
X_valid = valid_color / 255
X_test  = test_color  / 255

y_train = np_utils.to_categorical(labels)
y_valid = np_utils.to_categorical(valid_labels)
num_classes = y_valid.shape[1]

# **Model & Evaluate**

Model 1

In [ ]:
def model1 ():
  
	model = Sequential()
	model.add(Convolution2D(30, 5, 5, padding="same", input_shape=(64, 64, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(15, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
 
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
model1 = model1()
history = model1.fit(X_train, y_train, validation_data=(X_valid, y_valid),epochs=20)
scores = model1.evaluate(X_valid, y_valid)

Epoch 1/20
50/50 [==============================] - 2s 19ms/step - loss: 0.6945 - accuracy: 0.5104 - val_loss: 0.6921 - val_accuracy: 0.5100
Epoch 2/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6924 - accuracy: 0.5296 - val_loss: 0.6893 - val_accuracy: 0.5500
Epoch 3/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6929 - accuracy: 0.5121 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6903 - accuracy: 0.5349 - val_loss: 0.6876 - val_accuracy: 0.5425
Epoch 5/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6835 - accuracy: 0.5298 - val_loss: 0.6734 - val_accuracy: 0.6075
Epoch 6/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6797 - accuracy: 0.5599 - val_loss: 0.6705 - val_accuracy: 0.5825
Epoch 7/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6652 - accuracy: 0.5917 - val_loss: 0.6657 - val_accuracy: 0.6125
Epoch 8/20
50

In [ ]:
scores1 = model1.evaluate(X_valid, y_valid, verbose=0)
print("Classification Error: %.2f%%" % (100-scores1[1]*100))

Classification Error: 39.00%


Model 2

In [ ]:
def model2 ():
  
	model = Sequential()
	model.add(Convolution2D(30, 5, 5, padding="same", input_shape=(64, 64, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(15, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='sigmoid'))
 
	model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) 
	return model

In [ ]:
model2 = model2()
history2 = model2.fit(X_train, y_train, validation_data=(X_valid, y_valid),epochs=20)
scores = model2.evaluate(X_valid, y_valid)

Epoch 1/20
50/50 [==============================] - 1s 14ms/step - loss: 0.6946 - accuracy: 0.4942 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 2/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6928 - accuracy: 0.5173 - val_loss: 0.6918 - val_accuracy: 0.5375
Epoch 3/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6921 - accuracy: 0.5246 - val_loss: 0.6928 - val_accuracy: 0.5100
Epoch 4/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6922 - accuracy: 0.5123 - val_loss: 0.6911 - val_accuracy: 0.5475
Epoch 5/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6933 - accuracy: 0.4964 - val_loss: 0.6914 - val_accuracy: 0.5275
Epoch 6/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6923 - accuracy: 0.5196 - val_loss: 0.6901 - val_accuracy: 0.5525
Epoch 7/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6896 - accuracy: 0.5400 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 8/20
50

In [ ]:
scores2 = model2.evaluate(X_valid, y_valid, verbose=0)
print("Classification Error: %.2f%%" % (100-scores2[1]*100))

Classification Error: 39.75%


Model 3

In [ ]:
def model3 ():
  
	model = Sequential()
	model.add(Convolution2D(30, 5, 5, padding="same", input_shape=(64, 64, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(15, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
 
	model.compile(loss='binary_crossentropy', optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01), metrics = ['accuracy']) 
	return model

In [ ]:
model3 = model3()
history3 = model3.fit(X_train, y_train, validation_data=(X_valid, y_valid),epochs=20)
scores3 = model3.evaluate(X_valid, y_valid)

Epoch 1/20
50/50 [==============================] - 1s 14ms/step - loss: 0.6982 - accuracy: 0.5018 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6944 - accuracy: 0.4851 - val_loss: 0.6931 - val_accuracy: 0.5050
Epoch 3/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6943 - accuracy: 0.4698 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6934 - accuracy: 0.4971 - val_loss: 0.6931 - val_accuracy: 0.5025
Epoch 5/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6942 - accuracy: 0.5078 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6933 - accuracy: 0.5059 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 7/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6932 - accuracy: 0.4943 - val_loss: 0.6928 - val_accuracy: 0.5625
Epoch 8/20
50

In [ ]:
scores3 = model3.evaluate(X_valid, y_valid, verbose=0)
print("Classification Error: %.2f%%" % (100-scores3[1]*100))

Classification Error: 43.00%


Model 4

In [ ]:
def model4 ():
  
	model = Sequential()
	model.add(Convolution2D(30, 5, 5, padding="same", input_shape=(64, 64, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(15, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
 
	model.compile(loss='binary_crossentropy', optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01), metrics = ['accuracy']) 
	return model

In [ ]:
model4 = model4()
history4 = model4.fit(X_train, y_train, validation_data=(X_valid, y_valid),epochs=20)
scores4 = model4.evaluate(X_valid, y_valid)

Epoch 1/20
50/50 [==============================] - 1s 14ms/step - loss: 0.6962 - accuracy: 0.4869 - val_loss: 0.6985 - val_accuracy: 0.5000
Epoch 2/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6980 - accuracy: 0.4810 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6939 - accuracy: 0.4609 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6934 - accuracy: 0.4987 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6936 - accuracy: 0.4839 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 6/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6943 - accuracy: 0.4677 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 7/20
50/50 [==============================] - 1s 10ms/step - loss: 0.6928 - accuracy: 0.5096 - val_loss: 0.6922 - val_accuracy: 0.5325
Epoch 8/20
50

In [ ]:
scores4 = model4.evaluate(X_valid, y_valid, verbose=0)
print("Classification Error: %.2f%%" % (100-scores4[1]*100))

Classification Error: 43.25%


# **Conclusion**

Four models have been created and trained with various optimizers, activation functions, and droupout rates, while epoch is set up 30 for all of them. However, the highest accuracy rate is only 60.25%. It seems like more researches are needed to achieve higher accuracy. 